# colab版MMD自動トレースv2へようこそ！(実行編)

# 始めに

このツールの稼働状況やメンテナンス情報はTwitter（[@miu200521358](https://twitter.com/miu200521358)）にて行っています。

エラーになる、起動しない、などの場合、まずは現在の配布状況をご確認ください。

リプやDM等でのお問い合わせも受け付けています。

また、v2では会員登録後にダウンロードできる学習データが必要なため、会員登録をお願いしています。

# 目次

このノートブックでは、MMD自動トレースの準備と実行を行います。

ノートブックの使い方にはクセがあるので、「[準備編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)」で使い方に慣れてください。

画面左上の「＝」（実際は三本線）をクリックして下さい。目次が開きます。（既に開いている場合は次へ進んでください）

![目次](https://drive.google.com/uc?export=view&id=1HGk4sJmcPtMbMwcJOvE3aU1GjvKinwA_)


ノートブックを上から順に確認し、以下手順をひとつずつ実行してください。

実行が必要なセルには番号を振ってあります。①から順番に実行してください。


- **「①　環境設定」**
  - ランタイムがGPUに変更できたことを確認します
    - 変更のやり方は、導入編をご確認ください
  - Tensorflowのバージョンを1.xに変更します
  - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロードします
   - 準備や実際のトレース処理等、長い処理時に鳴らします
   - 不要の場合は、ブラウザの音量をミュートにしてください
- **「②　Googleドライブ・SDKとの連携」**
  - GoogleドライブまたSDK（ツール）と連携します
  - 連携のやり方は、導入編をご確認ください
- **「③　準備」**
    - MMD自動トレースに必要なコードをcolab上に構築します。
    - 大体10分くらいかかります。
- **「④　データDL・配置」**
    - MMD自動トレースに必要な学習データのうち、個別にDLしていただく必要のあるデータを配置します。
- **「⑤　動画配置」**
    - MMD自動トレースの対象となる映像データを配置します。
- **「⑥　MMD自動トレース実行」**
    - MMD自動トレースを実行します。

---

※作業中に下記のような警告が出ることがありますが、そのまま進めていただいて問題ありません

![警告](https://drive.google.com/uc?export=view&id=1mRW32urnPQ4LS4xrLEoPdp_XCqlq1HUF)


# MMD自動トレース実行

## **①　環境設定**

ヘッダの "ランタイム"　＞　"ランタイムのタイプを変更"　＞　"GPU"　を選択して下さい。

変更できたら、下のセルを実行して下さい。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①】　
#@markdown - ランタイムがGPUであることの確認
#@markdown - Tensorflowのバージョンを 1.x に変更
#@markdown - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロード

#@markdown ■■■■■■■■■■■■■■■■■■

exec_dict = {}
exec_dict['1'] = True

! echo --------------
! echo 【A】 ランタイムをGPUに変更
! echo --------------

! nvcc --version
! nvidia-smi

! echo --------------
! echo 【B】 Tensorflowのバージョンを 1.x に変更
! echo --------------

%tensorflow_version 1.x

%tensorflow_version

! echo --------------
! echo 【C】 効果音をダウンロード
! echo --------------

! wget --no-check-certificate -c "https://soundeffect-lab.info/sound/anime/mp3/sceneswitch1.mp3"

from IPython.display import Audio, display
def play_sound():
    try:
        display(Audio("sceneswitch1.mp3", autoplay=True))
    except:
        print("■■■■■■■■■■■■■■■")
        print("■　効果音が再生できませんでした")
        print("■■■■■■■■■■■■■■■")

play_sound()


**【OK】**

下記のように表示されて、最後に音が鳴ったら成功です。

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1WK0Oo3qQESB8hVlkvyWfsq17LUn3N308)

---

**【NG】**

下記のように表示されていたら、ランタイムの変更に失敗しているので、導入編を再度確認して、ランタイムを変更してください。

![GPU変更成功](https://drive.google.com/uc?export=view&id=1E230eOWMctbD2F7qTvn_fR-3gBRQQ_uQ)


## ②　Googleドライブ・SDKとの連携

Googleドライブの `autotrace` フォルダと連携します。

下のAとBのセルを順番に実行してください。

### ②-A Googleドライブとの連携

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル②-A】　Googleドライブとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

if os.path.exists(base_path):
    ! echo "autotraceフォルダの中身 -----------"
    ! ls -l "$base_path"
    ! echo "--------------------"

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　autotraceフォルダとの連携が成功しました。")
    print("■■■■■■■■■■■■■■■")

    exec_dict['2-A'] = True
else:
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotraceフォルダがGoogleドライブの直下に見つかりませんでした。")
    print("■　準備編から実行し直してください。")
    print("■■■■■■■■■■■■■■■")

play_sound()

**【OK】**

下記のように、「autotrace」フォルダの中身が表示されていたら成功です。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=19OhLvcyP-CN90KWDmkmBuSRR3BpO5GzV)

---

**【NG】**

下記のように、エラーが出ていたら、セル①が実行されていません。

もしくは環境が既に破棄されています。

目次からセル①に戻って、実行し直してください。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=1Tsjzs2QAECPMfUTaTh7PYX2cMEypEIF2)

miuのGoogleドライブから必要なデータをDLするため、`GoogleSDK` と連携します。

セル②-Aと同じく、URLからアクセス許可を与えてください。

下のセルを実行してください。

### ②-B GoogleSDKとの連携

In [ ]:


#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル②-B】　GoogleSDKとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

# Googleドライブアクセスライブラリ
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['2-B'] = True

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　Google SDK との連携が成功しました。")
print("■■■■■■■■■■■■■■■")

play_sound()

**【OK】**

下記のようにメッセージが表示されていたら成功です。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=1NiBFAK47dJLEcTagWGQrrHgnpKzsudvt)


## **③　準備**

MMD自動トレースコードをColab上に構築します。

下のセルを実行してください。

たくさんメッセージが出ます。

セル①でDLした音が鳴ったら終了です。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル③】　コード準備実行

#@markdown ■■■■■■■■■■■■■■■■■■

exec_dict['3'] = True

# MMD自動トレースキットバージョンタグ
ver_tag = "work_v1.00"

! git clone  --depth 1 -b "$ver_tag" "https://github.com/miu200521358/expose_mmd.git"

! pip install -r "expose_mmd/requirements.txt"

import os 
os.environ["CUDA_HOME"]="/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"]="/usr/local/cuda/lib64"

! cd expose_mmd/lighttrack/lib/ && python setup.py install
! cd expose_mmd/lighttrack/graph/torchlight && python setup.py install

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　コードの構築が完了しました。")
print("■■■■■■■■■■■■■■■")

play_sound()

## **④　データDL・配置**

人物推定に必要なデータを2件、それぞれのサイトに会員登録して、ダウンロードしてください。

1. [ExPoseプロジェクトページ](https://expose.is.tue.mpg.de) にアクセスしてください。

2. 「SignIn」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1__io7EjqFQnhq08Zi_YfxKUeSottZWJl)

3. 「Register」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1B8mJPJuTdHANo0UJ71Gue6CmznSDCdbw)

4. 「メールアドレス」「パスワード」を入力して、スイッチをすべてONにして、「Register」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1R2jCZHKJu71hXLW2KHhSLswDjwJEWEiZ)

5. 届いたメールのリンクをクリックしてください。

6. 登録した「メールアドレス」「パスワード」を入力して、「LogIn」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1loay_bPetEBQfaOQIpS_JCLhZrFHolyF)

7. 「Download」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1GFhczKTl5Mc6E6PSRXT0hznCVfo-FWDr)

8. 画面中程の「ExPose Model」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1FjaHb9yGr52nI27xBLHAQBBz4gOEnR6x)

9. ダウンロードしたzipファイル（expose_data.zip）を、`autotrace` フォルダに置いてください。

10. 下のセルを実行してください。



### ④-A ExPose学習データ

In [ ]:


#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル④-A】　expose_data の配置

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['4-A'] = True

expose_data_path = f"{base_path}/expose_data.zip"

if not os.path.exists(expose_data_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に expose_data.zip が見つかりませんでした。")
    print("■■■■■■■■■■■■■■■")
else:
    ! cp "$expose_data_path" . && unzip ./expose_data.zip && mv data expose_mmd/

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　expose_data の配置が成功しました。")
print("■■■■■■■■■■■■■■■")

play_sound()

1. [SMPL-Xプロジェクトページ](https://smpl-x.is.tue.mpg.de) にアクセスしてください。

2. Exposeと同様に会員登録をしてください。

3. 画面上部の「download SMPL-X v1.1」ボタンをクリックしてください。

![SignIn](https://drive.google.com/uc?export=view&id=1cGOHUdhDtSOfIqIHYUXhQTc5ki5pgl85)

4. ダウンロードしたzipファイル（models_smplx_v1_1.zip）を、`autotrace` フォルダに置いてください。

5. 下のセルを実行してください。

### ④-B SMPL-X学習データ

In [ ]:

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル④-B】　SMPL-Xデータ の配置

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['4-B'] = True

smplx_data_path = f"{base_path}/models_smplx_v1_1.zip"

if not os.path.exists(smplx_data_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に models_smplx_v1_1.zip が見つかりませんでした。")
    print("■■■■■■■■■■■■■■■")
else:
    ! cp "$smplx_data_path" . && unzip ./models_smplx_v1_1.zip && mv ./models expose_mmd/data

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　models_smplx_v1_1 の配置が成功しました。")
print("■■■■■■■■■■■■■■■")

play_sound()

### ④-C その他の学習データ

下のセルを実行してください。色々とメッセージが出た後、音が鳴ります。

In [ ]:

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル④-C】　その他のデータの配置

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['4-C'] = True

! cd expose_mmd/lighttrack/weights/ && sh download_weights.sh
! cd expose_mmd/data/ && wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 && bunzip2 shape_predictor_68_face_landmarks.dat.bz2

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　その他のデータ の配置が成功しました。")
print("■■■■■■■■■■■■■■■")

play_sound()

## **⑤　入力映像アップロード**

処理したい映像ファイルを、準備してください。

 - ファイル名は **半角英数字のみ** にしてください。opencvは2バイト文字を読み込めません。
 - Googleドライブの **autotrace** フォルダ 直下に置いてください。
 - **マウント後のGooleドライブ上のファイルの上書きや更新は正しく認識されません。** 新しいファイルは新規の名前でアップロードしてから処理して下さい。
 - `autotrace` フォルダの直下にアップロードしてください。
 - 下のセルを実行して下さい。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑤】　入力映像ファイルアップロード

#@markdown ■■■■■■■■■■■■■■■■■■


#@markdown 解析対象となる映像のファイルの名前を入力して、セルを実行してください。

input_video_name = "45seconds.mp4"  #@param {type: "string"}

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['5'] = True

input_video_path = f"{base_path}/{input_video_name}"

if not os.path.exists(input_video_path):
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　autotrace 直下に 指定動画 が見つかりませんでした。")
    print("■　ファイルパス: " + input_video_path)
    print("■■■■■■■■■■■■■■■")
else:
    ! cp "$input_video_path" .

    input_video_path = f"/content/{input_video_name}"

    print("■■■■■■■■■■■■■■■")
    print("■　** OK **")
    print("■　入力動画 の設定が完了しました。")
    print("■　ファイルパス: " + input_video_path)
    print("■■■■■■■■■■■■■■■")

play_sound()

## **⑥　MMD自動トレース実行**

MMD自動トレースは人物推定・追跡指定・モーション出力に分けて実行していただきます。

下のセルを実行してください。

※下記メッセージが出てきたら、1回目は無視して大丈夫です。

2回目が出てきたら、入力映像をキリのいいところで区切って分けて処理してください。

![容量オーバー](https://drive.google.com/uc?export=view&id=1EKt3nCK6ZYjgkNoflQNzSnW_0s4WnJYO)




### ⑥-A　人物推定

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-A】　人物推定

#@markdown ■■■■■■■■■■■■■■■■■■

if not exec_dict or '1' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル①が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '2-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル②-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '3' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル③が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-A' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Aが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-B' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Bが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '4-C' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル④-Cが実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

if not exec_dict or '5' not in exec_dict.keys():
    print("■■■■■■■■■■■■■■■")
    print("■　** ERROR **")
    print("■　セル⑤が実行されていない可能性があります。")
    print("■　目次から戻って実行してください。")
    print("■■■■■■■■■■■■■■■")

exec_dict['6-A'] = True

! rm -r '/content/output'

import datetime
process_img_dir_name = "{0}_{1:%Y%m%d_%H%M%S}".format(os.path.basename(input_video_path).replace('.', '_'), datetime.datetime.now())
process_img_dir_path = os.path.join('/content/output', process_img_dir_name)
os.makedirs(process_img_dir_path, exist_ok=True)

! cd expose_mmd/ && python executor.py --video-file "$input_video_path" --parent-dir "$process_img_dir_path" --process prepare,expose,tracking --verbose 20 --log-mode 0

! rm './out.webm'

# mp4に変換
bbox_file_path = f"{process_img_dir_path}/bbox.mp4"

!ffmpeg -i "$bbox_file_path" -vcodec vp9 ./out.webm

play_sound()

⑥-Aで準備した結果を、以下の観点で確認します。

 - 人物であろうと認識された箇所に枠が入ります。
 - 一人の人物を追いかける枠の左上にあるINDEXを確認してください。
 - 現在のフレーム番号は左上に表示しています。
 - 綺麗に人物が取れていないところは、枠が取れていても除外した方が綺麗になります。
 - 指定方法
   - INDEXを最初から最後まで取り入れる場合：[INDEX番号]
   - INDEXを最初から途中まで取り入れる場合：[INDEX番号]:-[終了フレーム]
   - INDEXを途中から最後まで取り入れる場合：[INDEX番号]:[開始フレーム]-
   - INDEXを途中から途中まで取り入れる場合：[INDEX番号]:[開始フレーム]-[終了フレーム]

INDEX指定例

 ```
0,44:263-,94,126,145,148,159,161:-3858,182,244,246
1,123,157,163:3572-,170,180,208:4427-,213,239:5580-
2,44:-261,49,61,64,71,95,135,139,146,156,188,197,205,215,247
 ```

 下のセルを実行してください。

### ⑥-B　人物推定結果確認

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-B】　人物推定結果確認

#@markdown ■■■■■■■■■■■■■■■■■■

from IPython.display import HTML
import base64
import io

def play_video(bbox_file_path):
    video = io.open(bbox_file_path, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="1280" height="720" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))

play_video('./out.webm')

### ⑥-C　人物追跡指定

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-C】　人物追跡指定

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 人物がひとりずつ動画の最後まで繋がるように、INDEXを指定して下さい。

#@markdown 指定が終わったら、セルを実行してください。このセルは繰り返し実行できます。

order_1 = "16,92,114,308,355"  #@param {type: "string"}

order_2 = "18,179,210,238,312"  #@param {type: "string"}

order_3 = ""  #@param {type: "string"}

order_4 = ""  #@param {type: "string"}

order_5 = ""  #@param {type: "string"}

order_6 = ""  #@param {type: "string"}

order_7 = ""  #@param {type: "string"}

order_8 = ""  #@param {type: "string"}

order_9 = ""  #@param {type: "string"}

order_data = [order_1, order_2, order_3, order_4, order_5, order_6, order_7, order_8, order_9]
order_file_path = f'{process_img_dir_path}/order.csv'

with open(order_file_path, 'w') as f:
    for order in order_data:
        f.write(order)
        f.write("\n")

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　人物INDEXの指定が成功しました。")
print("■　ファイルパス: " + order_file_path)
print("■■■■■■■■■■■■■■■")

play_sound()

### ⑥-D　追跡指定実行

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-D】　追跡指定実行

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 先ほど指定したINDEXで人物追跡を実行します。。

#@markdown このセルを実行してください。このセルは繰り返し実行できます。

! cd expose_mmd/ && python executor.py --img-dir "$process_img_dir_path" --order-file "$order_file_path" --process order --verbose 20 --log-mode 0

# mp4に変換
ordered_file_path = f"{process_img_dir_path}/ordered_bbox.avi"

! rm './out2.webm'

!ffmpeg -i "$ordered_file_path" -vcodec vp9 ./out2.webm

play_sound()

### ⑥-E　追跡指定確認

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-E】　追跡指定確認

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 意図した通りに人物を追跡出来ているか確認します。

#@markdown このセルを実行してください。

#@markdown 想定通りではなかった場合、⑥-Cに戻ってINDEXを指定し直してください。

from IPython.display import HTML
import base64
import io

def play_video(bbox_file_path):
    video = io.open(bbox_file_path, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="640" height="360" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))

play_video('./out2.webm')

### ⑥-F　モーション出力

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥-F】　モーション出力

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 実行したい任意機能にチェックを入れてください。

#@markdown このセルは繰り返し実行できます。

#@markdown **表情認識**　（かなり時間がかかります）

process_face = False #@param {type:"boolean"}

#@markdown **身体モーション出力**　（表情のみを出力したい場合などはOFFにしてください）

process_motion_body = True #@param {type:"boolean"}

#@markdown **上半身のみのモーション出力**　（センターYを出力しません）

process_motion_upper = False #@param {type:"boolean"}

#@markdown **手モーション出力**　（やや時間がかかります）

process_motion_hand = False #@param {type:"boolean"}

#@markdown **顔モーション出力**　（やや時間がかかります）

process_motion_face = False #@param {type:"boolean"}

#@markdown チェックできたら、このセルを実行してください。

#@markdown Googleドライブの `autotrace`フォルダの下に「`<動画ファイル名>_<実行年月日>_<実行時分秒（米国時間））>`」のフォルダの下に、VMDファイルとAVIファイルを出力しているので、それをダウンロードしてください。

process_txt = "smooth,motion"
if process_face:
    process_txt = "face,smooth,motion"

motion_body_flg = "1" if process_motion_body else "0"
motion_hand_flg = "1" if process_motion_hand else "0"
motion_face_flg = "1" if process_motion_face else "0"
motion_upper_flg = "1" if process_motion_upper else "0"

! cd expose_mmd/ && python executor.py --img-dir "$process_img_dir_path" --order-file "$order_file_path" --process $process_txt --body-motion $motion_body_flg --hand-motion $motion_hand_flg --face-motion $motion_face_flg --upper-motion $motion_upper_flg --verbose 20 --log-mode 0

import shutil

# 日本語対策でpythonコピー
output_name = os.path.basename(input_video_name).replace('.', '_')
output_path = os.path.join(base_path, process_img_dir_name)

if not os.path.exists(output_path):
    shutil.copytree(f'{process_img_dir_path}/motion', output_path)
else:
    import glob
    for vmd_path in glob.glob(os.path.join(process_img_dir_path, "motion", "*.vmd")):
        out_vmd_path = os.path.join(output_path, os.path.basename(vmd_path))
        shutil.copy(vmd_path, out_vmd_path)

shutil.copy(f'{process_img_dir_path}/ordered_bbox.avi', output_path)

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　モーションの出力が成功しました。")
print("■　ファイルパス: " + output_path)
print("■■■■■■■■■■■■■■■")

play_sound()


# 自動トレース後の作業

## ①　GoogleドライブにVMDが出力されない場合

 - **ERROR**　**CRITICAL**　というメッセージが出ていないか確認してください。
 - メッセージが出ていない場合、処理出力フォルダにVMDファイルが出力されているか確認してください。
   1. 画面左のフォルダーボタンをクリックしてください。

 ![フォルダーボタン](https://drive.google.com/uc?export=view&id=1AoTQjFHaz2uY4jGJttmVRkWk2nct2kb3)

   2. `output/<動画ファイル名>_<実行年月日>_<実行時分秒>/motion` フォルダの下にファイルが出来ているか確認してください。

 ![motionフォルダ](https://drive.google.com/uc?export=view&id=1wJsomaHWFwSyt6lmt7Zuli8AJ-cxi3uX)

     - VMDファイルがある場合、それをダウンロードしてください。
     - VMDファイルがない場合、どこかでエラーが発生しています。
     - エラーの原因が分からない場合は、ノートブックを共有してください。
       - 共有方法は、「[準備編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)」を確認してください。


## ② 　関節位置データが欲しい場合

関節位置データ（姿勢推定・人物追跡・表情認識の結果データ）が必要な場合、下のセルを実行すると、Googleドライブに該当データがコピーされるので、それをダウンロードしてください。

In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 生データが欲しい場合、このセルを実行してください。

#@markdown ■■■■■■■■■■■■■■■■■■

json_path = os.path.join(output_path, 'ordered')

import shutil
shutil.copytree(f'{process_img_dir_path}/ordered', json_path)

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　生データのGoogleドライブへのコピーが完了しました。")
print("■　フォルダパス: " + json_path)
print("■■■■■■■■■■■■■■■")

play_sound()


In [ ]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown スムージング後データが欲しい場合、このセルを実行してください。

#@markdown ■■■■■■■■■■■■■■■■■■

json_path = os.path.join(output_path, 'smooth')

import shutil
shutil.copytree(f'{process_img_dir_path}/smooth', json_path)

print("■■■■■■■■■■■■■■■")
print("■　** OK **")
print("■　スムージング後データのGoogleドライブへのコピーが完了しました。")
print("■　フォルダパス: " + json_path)
print("■■■■■■■■■■■■■■■")

play_sound()


## ③　全打ちデータから、不要キーを除きたい場合

[モーションサポーター](https://seiga.nicovideo.jp/seiga/im10581987) の「スムージング」を使用してください。

**オススメ設定**

 1. 「処理対象：センターとグルーブ」「処理回数：10」「補間方法：補間曲線に従う」でスムージング
 2. 「処理対象：全ボーン・フレーム」「処理回数：3」「補間方法：補間曲線に従う」でスムージング


## ④　奥行きを付けたい場合

全親で大体の奥行き（必要であればセンター位置の補正も）を入力した後、[モーションサポーター](https://seiga.nicovideo.jp/seiga/im10581987) の「全親移植」を使用してください。

## ⑤　あにまさミクから他のモデルに合わせたモーションデータに変換したい場合

[VMDサイジング](https://seiga.nicovideo.jp/seiga/im9755721) を使用してください。

**オススメ設定**

1. 「元モデル：あにまさ式ミク準標準」「スタンス追加補正：ON」「捩り分散：ON」でお好きなモデルに合わせてサイジング

**オススメの作業順番**

1. スムージング③（あにまさミク）
2. 全親移植④（あにまさミク）
3. サイジング⑤（任意のモデル）
4. モーション修正（任意のモデル）


## ⑥　最初からやり直したい場合

1. メニューの「ランタイム」＞「ランタイムを出荷状態にリセット」＞「はい」をクリックしてください。

 ![リセット](https://drive.google.com/uc?export=view&id=1Eyi6OLpndFo63tyJsMtukIiL9p4wqBJs)

2. メニュー→の「再接続」をクリックしてください。

 ![再接続](https://drive.google.com/uc?export=view&id=1h7jF222tU4ZQz_jIMeDKYb9CNIbHTN1k)

3. 緑のチェックマークがついたらリセット完了です。

 ![リセット完了](https://drive.google.com/uc?export=view&id=1FXq971EddklGtHMHpm_vQM0MtRvQrJ-y)


# ライセンス・クレジット

MMD自動トレースの結果を公開・配布する場合は、必ずライセンスのご確認をお願い致します。Unity等他への適用の場合も同様です。

ライセンスを記載いただけたらとても有難いです。

[MMD自動トレースv2 ライセンス・クレジット](https://ch.nicovideo.jp/miu200521358/blomaga/ar1974413)